In [17]:
import numpy as np
from tqdm.notebook import trange
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot

import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [18]:
import chess
import chess.pgn

class State:
  def __init__(self, board=None):
    if board is None:
      self.board = chess.Board()
    else:
      self.board = board

  def serialize(self):
    serialized_board = np.zeros(64, np.uint8)
    values = {"P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6, \
              "p": 9, "n":10, "b":11, "r":12, "q":13, "k": 14}

    for i in range(64):
      piece = self.board.piece_at(i)
      if piece is not None:
        serialized_board[i] = values[piece.symbol()]
        # TODO: handle special stuff (castling and en passaint)

    return serialized_board

  def edges(self):
    return list(self.board.legal_moves)

  # makes 
  def to_net_input(self):
    s_board = self.serialize()
    # TODO: pass info about the game, such as who's turn it is, etc
    return s_board


In [19]:
data_path = "../data/preprocessed/full_dataset.npz"

data = np.load(data_path)
X_train, Yt = data.f.arr_0, data.f.arr_1

Y_train = []
for y in Yt:
  Y_train.append([y])
Y_train = np.array(Y_train)

print(X_train.shape)
print(Y_train.shape)
print(X_train)
print(Y_train)

(462295, 64)
(462295, 1)
[[ 4  2  3 ... 11 10 12]
 [ 4  2  3 ... 11 10 12]
 [ 4  2  3 ... 11 10 12]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [20]:
class ValueNet(nn.Module):
  def __init__(self, in_len, out_len):
    super(ValueNet, self).__init__()
    
    self.relu = nn.ReLU()
    
    self.fc1 = nn.Linear(in_len, 128)
    self.bn1 = nn.BatchNorm1d(128)
    self.fc2 = nn.Linear(128, 128)
    self.bn2 = nn.BatchNorm1d(128)
    self.fc3 = nn.Linear(128, out_len)

  def forward(self, x):
    x = self.relu(self.bn1(self.fc1(x)))
    x = self.relu(self.bn2(self.fc2(x)))
    x = torch.tanh(self.fc3(x))
    return x

  def predict(self, x):
    return F.sigmoid(self(x))

def save_model(model, path):
  torch.save(model.state_dict(), path)
  print("[+] Model saved at:", path)

def load_model(model, path, val=False):
  model.load_state_dict(torch.load_state_dict(torch.load(path)))
  if val:
    model.eval()
  else:
    model.train()
  print("[+] Loaded model from:", path)
  

model = ValueNet(X_train.shape[1], 1).to(device)
print(model)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [22]:
model_path = "../models/chess_model.pth"

def train(model, X_train, Y_train, X_test=None, Y_test=None):
  model.train()
  
  lr = 1e-3
  #epochs = 100
  epochs = 10
  BS = 128

  losses, accuracies = [], []

  loss_func = nn.BCELoss()# TODO: different loss (regression at [-1,1])
  optim = torch.optim.Adam(model.parameters(), lr=lr)

  for epoch in range(epochs):
    print("[+] Epoch", epoch+1)
    epoch_losses = []
    epoch_acc = []# TODO: no accuracies!!!
    for i in (t := trange(0, len(X_train), BS)):
      X = torch.tensor(X_train[i:i+BS]).float().to(device)
      Y = torch.tensor(Y_train[i:i+BS]).float().to(device)

      optim.zero_grad()
      out = model(X)
      cat = torch.round(out)
      accuracy = (cat == Y).float().mean()
      loss = loss_func(out, Y).mean()
      loss.backward()
      optim.step()

      # TODO: stats (add tensorboard as well)
      epoch_acc.append(accuracy.item())
      epoch_losses.append(loss.item())
      epoch_acc.append(accuracy.item())
      t.set_description("loss %.2f, acc %.2f"%(loss, accuracy))
      
    avg_acc = np.array(epoch_acc).mean()
    avg_loss = np.array(epoch_losses).mean()
    print("[~] Avg Epoch Loss %.2f - Accuracy %.2f"%(avg_acc, avg_loss))
    accuracies.append(avg_acc)
    losses.append(avg_loss)

  print("[+] Training Done!")
  plt.plot(losses)
  plt.plot(accuracies)
  plt.show()
      
      
train(model, X_train, Y_train)

[+] Epoch 1


  0%|          | 0/3612 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
def eval(model, X_test, Y_test):
  model.eval()

eval(model, [], [])